# Text AI Generation for Twitter Bot

## Setup
### Import TensorFlow and other libraries

In [18]:
import tensorflow as tf

import numpy as np
import os
import time

### Read the data

In [19]:
PATH_TO_FILE = "./datasets/xkaubu_pure.txt"

# Read, then decode for py2 compat
text = open(PATH_TO_FILE, "rb").read().decode(encoding="utf-8")

# Length of text is the number of characters in it
print(f"Length of text: {len(text)} characters")

Length of text: 93736 characters


In [20]:
# Take a look at the first 250 characters in text
print(text[:250])

Don't put a space between "therapist" 😱😱
Worst mistake or my
rutracker is literally based in russia...
new AOT episode!!
I've got 400 lines or so of 357 of my 2955 tweets
is this that minecraft painting
I can kinda pronounce it
YOOOO this is an epic 


In [21]:
# The unique characters in the file
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

170 unique characters


## Process the text
### Vectorize the text

In [22]:
example_texts = ["abcdefg", "xyz"]

chars = tf.strings.unicode_split(
    example_texts,
    input_encoding="UTF-8"
)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [23]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

# It converts from tokens to character IDs:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[62, 63, 64, 65, 66, 67, 68], [85, 86, 87]]>

In [24]:
# To convert the ids back into characters
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [25]:
# Join the characters back into strings
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [26]:
def text_from_ids(ids):
    return tf.strings.reduce_join(
        chars_from_ids(ids),
        axis=-1
    )

### Create training examples and targets

In [28]:
all_ids = ids_from_chars(
    tf.strings.unicode_split(
        text,
        "UTF-8"
    )
)
all_ids

<tf.Tensor: shape=(93736,), dtype=int64, numpy=array([36, 76, 75, ...,  2, 70, 77])>

In [29]:
ids_dataset = tf.data.Dataset.from_tensor_slices(
    all_ids
)

In [30]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

D
o
n
'
t
 
p
u
t
 


In [31]:
SEQ_LENGTH = 100
examples_per_epoch = len(text)//(SEQ_LENGTH+1)

In [32]:
# Convert these individual characters to sequences of the desired size

sequences = ids_dataset.batch(
    SEQ_LENGTH+1,
    drop_remainder=True
)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'D' b'o' b'n' b"'" b't' b' ' b'p' b'u' b't' b' ' b'a' b' ' b's' b'p'
 b'a' b'c' b'e' b' ' b'b' b'e' b't' b'w' b'e' b'e' b'n' b' ' b'"' b't'
 b'h' b'e' b'r' b'a' b'p' b'i' b's' b't' b'"' b' ' b'\xf0\x9f\x98\xb1'
 b'\xf0\x9f\x98\xb1' b'\n' b'W' b'o' b'r' b's' b't' b' ' b'm' b'i' b's'
 b't' b'a' b'k' b'e' b' ' b'o' b'r' b' ' b'm' b'y' b'\n' b'r' b'u' b't'
 b'r' b'a' b'c' b'k' b'e' b'r' b' ' b'i' b's' b' ' b'l' b'i' b't' b'e'
 b'r' b'a' b'l' b'l' b'y' b' ' b'b' b'a' b's' b'e' b'd' b' ' b'i' b'n'
 b' ' b'r' b'u' b's' b's' b'i' b'a' b'.' b'.'], shape=(101,), dtype=string)


In [33]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'Don\'t put a space between "therapist" \xf0\x9f\x98\xb1\xf0\x9f\x98\xb1\nWorst mistake or my\nrutracker is literally based in russia..'
b".\nnew AOT episode!!\nI've got 400 lines or so of 357 of my 2955 tweets\nis this that minecraft painting"
b'\nI can kinda pronounce it\nYOOOO this is an epic chungus moment\nDo better\nBlocked and unfollowed\nfair '
b"enough\ni've been pronouncing it potat\ndo you pronounce ur name potato or potat?\n???\nGot some of my me"
b"ssages already\nSpunkbob:\nGreat insight! But don't think we'll forget about the Assange NFTs\nLMAO\ni'm "


In [34]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [35]:
split_input_target(
    list("Tensorflow")
)

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [36]:
dataset = sequences.map(split_input_target)

In [37]:
for input_example, target_example in dataset.take(1):
    print(f"Input: {text_from_ids(input_example).numpy()}")
    print(f"Target: {text_from_ids(target_example).numpy()}")

Input: b'Don\'t put a space between "therapist" \xf0\x9f\x98\xb1\xf0\x9f\x98\xb1\nWorst mistake or my\nrutracker is literally based in russia.'
Target: b'on\'t put a space between "therapist" \xf0\x9f\x98\xb1\xf0\x9f\x98\xb1\nWorst mistake or my\nrutracker is literally based in russia..'


### Create training batches

In [38]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite
# sequences, so it doesn't attempt to shuffle the entire
# sequence in memory. Instead, it maintains a buffer in
# which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [39]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [40]:
class MyModel(tf.keras.Model):
    def __init__(
        self, 
        vocab_size, 
        embedding_dim, 
        rnn_units
    ):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(
            vocab_size,
            embedding_dim
        )
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)
    
    def call(
        self,
        inputs,
        states=None,
        return_state=False,
        training=False
    ):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(
            x,
            initial_state=states,
            training=training
        )
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [41]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup`
    # layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

## Try the model

In [42]:
# Check the shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(f"{example_batch_predictions.shape}# (batch_size, sequence_length, vocab_size)")

2022-03-09 02:37:17.886856: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


(64, 100, 171)# (batch_size, sequence_length, vocab_size)


In [43]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  43776     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  175275    
                                                                 
Total params: 4,157,355
Trainable params: 4,157,355
Non-trainable params: 0
_________________________________________________________________


In [44]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],
    num_samples=1
)
sampled_indices = tf.squeeze(
    sampled_indices,
    axis=-1
).numpy()

In [45]:
sampled_indices

array([  0,  11, 116,   7, 114,   5,  21, 119,  10, 145,  87,  65, 162,
       165,  31,  32,  33,  42,  38,  38,  34, 151,  85,  55,  57, 169,
        13,  65,  88, 160, 101,  69,  14,   4,  42,  50, 149, 141,   5,
        97, 151,  50, 114,  49, 160, 123, 169, 132, 108,  89, 148, 131,
       142, 121,  90,  22, 126, 166,  22,  48, 112,  73, 169,  23,   3,
       161, 155,  91,  25, 102, 119, 154,  23,  82,  91,  71, 132,   5,
        81,   8,  46, 169,  86, 122,  43,   8, 140, 147,  19,  51, 166,
       101,  22, 109,  33, 116, 136,  19, 140, 125])

In [46]:
print(f"Input:\n{text_from_ids(input_example_batch[0].numpy())}\n")
print(f"Next Char Predictions:\n{text_from_ids(sampled_indices).numpy().decode('utf-8')}")

Input:
b'it on any calculator\nLol you do realize 0\xe2\x81\xb0 = 1 right? \xf0\x9f\x98\x82\xf0\x9f\x98\x82\xf0\x9f\x98\x82\nI was making an observation at your puny a'

Next Char Predictions:
[UNK]*⁰&”#4☹)😦zd🤬🥰?@AJFFB😵xWY🧐,d🤦पh-"JR😱😔#ʔ😵R”Q🤦🅱🧐😁ह😭🔥😠❗×5👋🥵5P’l🧐6!🤨🙄æ8भ☹🙂6uæj😁#t'N🧐y️K'😐😬2S🥵प5ाA⁰😊2😐👀


## Train the model
### Attach an optimizer, and a loss function

In [47]:
loss = tf.losses.SparseCategoricalCrossentropy(
    from_logits=True
)

In [48]:
example_batch_mean_loss = loss(
    target_example_batch,
    example_batch_predictions,
)
print(f"Prediction shape:\t\t{example_batch_predictions.shape} # (batch_size, sequence_length, vocab_size)")
print(f"Mean loss:\t\t\t{example_batch_mean_loss}")

Prediction shape:		(64, 100, 171) # (batch_size, sequence_length, vocab_size)
Mean loss:			5.141695499420166


In [49]:
tf.exp(example_batch_mean_loss).numpy()

171.00546

In [50]:
model.compile(
    optimizer="adam",
    loss=loss
)

### Configure checkpoints

In [51]:
# Directory where the checkpoints will be saved
checkpoint_dir="./models/xkaubu"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(
    checkpoint_dir,
    "ckpt_{epoch}"
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

### Execute the training

In [76]:
EPOCHS = 300

In [77]:
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

Epoch 1/300
14/14 [==============================] - 1s 97ms/step - loss: 0.0406
Epoch 2/300
14/14 [==============================] - 1s 94ms/step - loss: 0.0409
Epoch 3/300
14/14 [==============================] - 2s 89ms/step - loss: 0.0406
Epoch 4/300
14/14 [==============================] - 1s 92ms/step - loss: 0.0408
Epoch 5/300
14/14 [==============================] - 1s 91ms/step - loss: 0.0405
Epoch 6/300
14/14 [==============================] - 1s 88ms/step - loss: 0.0405
Epoch 7/300
14/14 [==============================] - 2s 100ms/step - loss: 0.0409
Epoch 8/300
14/14 [==============================] - 2s 99ms/step - loss: 0.0410
Epoch 9/300
14/14 [==============================] - 1s 96ms/step - loss: 0.0407
Epoch 10/300
14/14 [==============================] - 1s 91ms/step - loss: 0.0409
Epoch 11/300
14/14 [==============================] - 2s 104ms/step - loss: 0.0409
Epoch 12/300
14/14 [==============================] - 1s 99ms/step - loss: 0.0407
Epoch 13/300
14/14 [===

## Generate text

In [78]:
# Single step prediction
class OneStep(tf.keras.Model):
    def __init__(
        self,
        model,
        chars_from_ids,
        ids_from_chars,
        temperature=1.0
    ):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being
        # generated.
        skip_ids = self.ids_from_chars(
            ["[UNK]"]
        )[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index
            values=[-float("inf")]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[
                len(
                    ids_from_chars.get_vocabulary()
                )
            ]
        )
        self.prediction_mask = tf.sparse.to_dense(
            sparse_mask
        )
    
    @tf.function
    def generate_one_step(
        self,
        inputs,
        states=None
    ):
        # Convert strings to token IDs
        input_chars = tf.strings.unicode_split(
            inputs,
            "UTF-8"
        )
        input_ids = self.ids_from_chars(
            input_chars
        ).to_tensor()

        # Run the model
        # predicted_logits.shape is
        # [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids,
            states=states,
            return_state=True
        )

        # Only use the last prediction
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask:
        # prevent "[UNK]" from being generated
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs
        predicted_ids = tf.random.categorical(
            predicted_logits,
            num_samples=1
        )
        predicted_ids = tf.squeeze(
            predicted_ids,
            axis=-1
        )

        # Convert from token IDs to characters
        predicted_chars = self.chars_from_ids(
            predicted_ids
        )

        # Return the characters and model state
        return predicted_chars, states

In [79]:
one_step_model = OneStep(
    model,
    chars_from_ids,
    ids_from_chars
)

### Actual generation code

In [82]:
start = time.time()
states = None
# Seed ideas:
# I, You, I'm, Yo, Who, This, Why, 
next_char = tf.constant([input("Seed: ")])
result = [next_char]

# Default: 1000
CHARS_GEN = 100000

for n in range(CHARS_GEN):
    next_char, states = one_step_model.generate_one_step(
        next_char,
        states=states
    )
    result.append(next_char)

# output/ai{VERSION}_{GENERATED FILE NO}{_IF NEW EPOCH}.txt
OUT_FILE="output/ai3_5_N.txt"

result = tf.strings.join(result)
end = time.time()

with open(OUT_FILE, "a") as f:
    f.write(result[0].numpy().decode("utf-8"))

print(f"{result[0].numpy().decode('utf-8')}\n\n{'_'*80}\n")
print(f"Run time: {end-start}")